# Imports and Installations

In [ ]:
!pip install -r /root/shared/fenics_models/fenics_requirements.txt

In [7]:
import dolfinx

In [11]:
dir(dolfinx.nls.PETSc.SNES)

['ConvergedReason',
 'NormSchedule',
 'Type',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'appctx',
 'atol',
 'callConvergenceTest',
 'cancelMonitor',
 'classid',
 'comm',
 'compose',
 'computeFunction',
 'computeJacobian',
 'computeNGS',
 'computeObjective',
 'converged',
 'create',
 'createPython',
 'decRef',
 'destroy',
 'diverged',
 'dm',
 'fortran',
 'getAppCtx',
 'getAttr',
 'getClassId',
 'getClassName',
 'getComm',
 'getCompositeNumber',
 'getCompositeSNES',
 'getConvergedReason',
 'getConvergenceHistory',
 'getConvergenceTest',
 'getDM',
 'getDict',
 'getFASCoarseSolve',
 'getFASCycleSNES',
 'getFASInjection',
 'getFASInterpolation',
 'ge

In [1]:
from math import ceil
import time
import json
import numpy as np
import sys
sys.path.insert(0, '/root/shared/fenics_models')
import fenics_helpers

# Convergence Studies

## Mesh Size Convergence

Informal parameter search shows that greatest displacements occur at around 120 degrees - perform convergence study here.

In [6]:
fixed_params = {'C_1': 1.8, 'beam_angle': 90, 'density': 0.00102, 'g': 9.81, 'width': 40, 'length': 90,
                'kappa': 3000, 'rtol': 1e-3, 'atol': 1e-3, 'max_iter': 50, 'elem_order': 2, 
                'num_load_steps': 50}

# Parameter to vary in convergence study:
elem_size_list = [fixed_params['width']/num_elem_width for num_elem_width in (12,)]  #3, 5, 7, 9, 10, 11, 
 
# Perform convergence study:
results = {key: [] for key in ('end_disp', 'elem_size', 'num_elem', 'num_elem_width', 'num_elem_length', 'volume')}
for i, elem_size in enumerate(elem_size_list):
    
    NL, NW = ceil(fixed_params['length']/elem_size), ceil(fixed_params['width']/elem_size)

    print(f"Simulating Mesh {i+1}/{len(elem_size_list)} (Num elem = {NL*NW*NW})")
    results['num_elem_width'].append(NW)
    results['num_elem_length'].append(NL)
    results['elem_size'].append(elem_size)
    results['num_elem'].append(NL*NW*NW)

    mesh = fenics_helpers.create_cuboidal_mesh(fixed_params['length'], fixed_params['width'], NL, NW)

    u = fenics_helpers.simulate_neohookean_beam(mesh, **fixed_params)
    
    results['end_disp'].append(fenics_helpers.compute_end_displacement(u, mesh, fixed_params['width'], fixed_params['length']))
    results['volume'].append(fenics_helpers.compute_pre_and_postdeformation_volume(u, mesh))

results['fixed_params'] = fixed_params

# Save results to json:
with open('neohookean_beam_meshsize_convergence.json', 'w') as f:
    json.dump(results, f, indent=4)

Simulating Mesh 1/1 (Num elem = 3888)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


RuntimeError: Newton solver did not converge because maximum number of iterations reached

## Kappa Convergence

In [2]:
fixed_params = {'C_1': 1.67, 'density': 0.00102, 'g': 9.81, 'width': 40, 'length': 90,  
                'beam_angle': 120, 'num_elem_width': 2, 'num_elem_length': 2, 'rtol': 1e-3, 'atol': 1e-3, 
                'max_iter': 10, 'elem_order': 2, 'num_load_steps': 20}

#'num_elem_width': 10, 'num_elem_length': 27

# Mesh doesn't vary for different kappa values - create it once at the start:
mesh = fenics_helpers.create_cuboidal_mesh(fixed_params['length'], fixed_params['width'], 
                                           fixed_params['num_elem_length'], fixed_params['num_elem_width'])

# Parameter to vary in convergence study:
kappa_list = [250, 500, 750, 1000, 1250, 1500, 2000, 2500, 2750, 3000, 3250]

# Perform convergence study:
results = {key: [] for key in ('end_disp', 'volume', 'kappa')}
for i, kappa in enumerate(kappa_list):

    print(f"Simulating Mesh {i+1}/{len(kappa_list)} (Kappa = {kappa})")
    results['kappa'].append(kappa)

    # Compute displacement of beam:
    u = fenics_helpers.simulate_neohookean_beam(mesh, kappa=kappa, **fixed_params)
    
    # Compute displacement at end of beam and change in beam volume before and after deformation:
    results['end_disp'].append(fenics_helpers.compute_end_displacement(u, mesh, fixed_params['width'], fixed_params['length']))
    results['volume'].append(fenics_helpers.compute_pre_and_postdeformation_volume(u, mesh))

results['fixed_params'] = fixed_params

# Save results to json:
with open('neohookean_beam_kappa_convergence.json', 'w') as f:
    json.dump(results, f, indent=4)

NameError: name 'param_combos' is not defined

# Data Generation

## Training Data

In [17]:
fixed_params = {'density': 0.00102, 'g': 9.81, 'width': 40, 'length': 90,
                'num_elem_width': 10, 'num_elem_length': 27, 'kappa': 3000, 'rtol': 1e-3, 
                'atol': 1e-3, 'max_iter': 10, 'elem_order': 2, 'num_load_steps': 30}
mesh = fenics_helpers.create_cuboidal_mesh(fixed_params['length'], fixed_params['width'], 
                                           fixed_params['num_elem_length'], fixed_params['num_elem_width'])  

In [15]:
# Values to vary:
num_train_pts = 10
min_C_1, max_C_1 = 1.67, 6.67
C_1_list = [C_1 for C_1 in np.linspace(min_C_1, max_C_1, num_train_pts)]
min_beam_angle, max_beam_angle = 0, 180
beam_angle_list = [y for y in np.linspace(min_beam_angle, max_beam_angle, num_train_pts)]
param_combos = fenics_helpers.create_param_combos(C_1=C_1_list, beam_angle=beam_angle_list)

# Compute test data points:
results = {key: [] for key in ('C_1', 'beam_angle', 'end_disp', 't_solve', 'volume')}
for i, params in enumerate(param_combos):
    
    print(f"Simulation {i+1}/{len(param_combos)} (C_1 = {params['C_1']}, Beam angle = {params['beam_angle']})")
    results['C_1'].append(params['C_1'])
    results['beam_angle'].append(params['beam_angle'])

    # Compute displacement of beam:
    t_start = time.time()
    u = fenics_helpers.simulate_neohookean_beam(mesh, C_1=params['C_1'], beam_angle=params['beam_angle'], **fixed_params)
    t_solve = time.time() - t_start
    results['t_solve'].append(t_solve)
    print(f'Simulation took {t_solve/60:.2f} mins.\n')
    
    results['end_disp'].append(fenics_helpers.compute_end_displacement(u, mesh, fixed_params['width'], fixed_params['length']))
    results['volume'].append(fenics_helpers.compute_pre_and_postdeformation_volume(u, mesh))

results['fixed_params'] = fixed_params

# Save dictionary to JSON file:
with open("neohookean_beam_training_data.json", 'w') as f:
    json.dump(results, f, indent=4)

Simulation 1/100 (C_1 = 1.67, Beam angle = 0.0)
Simulation took 7.55 mins.

Simulation 2/100 (C_1 = 1.67, Beam angle = 20.0)
Simulation took 7.55 mins.

Simulation 3/100 (C_1 = 1.67, Beam angle = 40.0)


KeyboardInterrupt: 

## Test Data

In [18]:
# Values to vary:
num_test_pts = num_train_pts-1
delta_C_1 = (max_C_1-min_C_1)/num_test_pts
delta_angle = (max_beam_angle-min_beam_angle)/num_test_pts
C_1_list = [C_1 for C_1 in np.linspace(min_C_1+0.5*delta_C_1, max_C_1-0.5*delta_C_1, num_test_pts)]
beam_angle_list = \
[angle for angle in np.linspace(min_beam_angle+0.5*delta_angle, max_beam_angle-0.5*delta_angle, num_test_pts-1)]
param_combos = fenics_helpers.create_param_combos(C_1=C_1_list, beam_angle=beam_angle_list)

# Compute test data points:
results = {key: [] for key in ('C_1', 'beam_angle', 'end_disp', 't_solve', 'volume')}
for i, params in enumerate(param_combos):
    
    print(f"Simulation {i+1}/{len(param_combos)} (C_1 = {params['C_1']}, Beam angle = {params['beam_angle']})")
    results['C_1'].append(params['C_1'])
    results['beam_angle'].append(params['beam_angle'])

    # Compute displacement of beam:
    t_start = time.time()
    u = fenics_helpers.simulate_neohookean_beam(mesh, C_1=params['C_1'], beam_angle=params['beam_angle'], **fixed_params)
    t_solve = time.time() - t_start
    results['t_solve'].append(t_solve)
    print(f'Simulation took {t_solve/60:.2f} mins.\n')
    
    results['end_disp'].append(fenics_helpers.compute_end_displacement(u, mesh, fixed_params['width'], fixed_params['length']))
    results['volume'].append(fenics_helpers.compute_pre_and_postdeformation_volume(u, mesh))
    
results['fixed_params'] = fixed_params

# Save dictionary to JSON file:
with open("neohookean_beam_test_data.json", 'w') as f:
    json.dump(results, f, indent=4)

Simulation 1/72 (C_1 = 1.9477777777777776, Beam angle = 10.0)
Simulation took 8.36 mins.

Simulation 2/72 (C_1 = 1.9477777777777776, Beam angle = 32.85714285714286)
Simulation took 8.38 mins.

Simulation 3/72 (C_1 = 1.9477777777777776, Beam angle = 55.714285714285715)
Simulation took 8.78 mins.

Simulation 4/72 (C_1 = 1.9477777777777776, Beam angle = 78.57142857142857)
Simulation took 11.33 mins.

Simulation 5/72 (C_1 = 1.9477777777777776, Beam angle = 101.42857142857143)
Simulation took 13.21 mins.

Simulation 6/72 (C_1 = 1.9477777777777776, Beam angle = 124.28571428571429)
Simulation took 12.82 mins.

Simulation 7/72 (C_1 = 1.9477777777777776, Beam angle = 147.14285714285714)
Simulation took 13.38 mins.

Simulation 8/72 (C_1 = 1.9477777777777776, Beam angle = 170.0)
Simulation took 8.19 mins.

Simulation 9/72 (C_1 = 2.503333333333333, Beam angle = 10.0)
Simulation took 8.10 mins.

Simulation 10/72 (C_1 = 2.503333333333333, Beam angle = 32.85714285714286)
Simulation took 8.10 mins.

S